In [ ]:
Table: Customers
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| customer_id   | int     |
| name          | varchar |
| country       | varchar |
+---------------+---------+
customer_id is the primary key for this table.
This table contains information of the customers in the company.
 

Table: Product
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| product_id    | int     |
| description   | varchar |
| price         | int     |
+---------------+---------+
product_id is the primary key for this table.
This table contains information of the products in the company.
price is the product cost.
 

Table: Orders
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| order_id      | int     |
| customer_id   | int     |
| product_id    | int     |
| order_date    | date    |
| quantity      | int     |
+---------------+---------+
order_id is the primary key for this table.
This table contains information on customer orders.
customer_id is the id of the customer who bought "quantity" products with id "product_id".
Order_date is the date in format ('YYYY-MM-DD') when the order was shipped.
 

Write an SQL query to report the customer_id and customer_name of customers who have spent at least $100 in each 
month of June and July 2020.

Return the result table in any order.

The query result format is in the following example.

 
Customers
+--------------+-----------+-------------+
| customer_id  | name      | country     |
+--------------+-----------+-------------+
| 1            | Winston   | USA         |
| 2            | Jonathan  | Peru        |
| 3            | Moustafa  | Egypt       |
+--------------+-----------+-------------+

Product
+--------------+-------------+-------------+
| product_id   | description | price       |
+--------------+-------------+-------------+
| 10           | LC Phone    | 300         |
| 20           | LC T-Shirt  | 10          |
| 30           | LC Book     | 45          |
| 40           | LC Keychain | 2           |
+--------------+-------------+-------------+

Orders
+--------------+-------------+-------------+-------------+-----------+
| order_id     | customer_id | product_id  | order_date  | quantity  |
+--------------+-------------+-------------+-------------+-----------+
| 1            | 1           | 10          | 2020-06-10  | 1         |
| 2            | 1           | 20          | 2020-07-01  | 1         |
| 3            | 1           | 30          | 2020-07-08  | 2         |
| 4            | 2           | 10          | 2020-06-15  | 2         |
| 5            | 2           | 40          | 2020-07-01  | 10        |
| 6            | 3           | 20          | 2020-06-24  | 2         |
| 7            | 3           | 30          | 2020-06-25  | 2         |
| 9            | 3           | 30          | 2020-05-08  | 3         |
+--------------+-------------+-------------+-------------+-----------+

Result table:
+--------------+------------+
| customer_id  | name       |  
+--------------+------------+
| 1            | Winston    |
+--------------+------------+ 
Winston spent $300 (300 * 1) in June and $100 ( 10 * 1 + 45 * 2) in July 2020.
Jonathan spent $600 (300 * 2) in June and $20 ( 2 * 10) in July 2020.
Moustafa spent $110 (10 * 2 + 45 * 2) in June and $0 in July 2020.


In [ ]:
Trial 1


select sum(quantity*price)

where order_date BETWEEN "2020-06-01" and  "2020-07-31"
group by customer_id, order_date


In [ ]:
Trial 2

This component finds sum(product*price) per customer_id. But the problem asks to find the sum per customer_id and per
months of June and July


select sum(o.quantity*p.price)
from Customers c inner join 
Orders o inner join
Product p
on c.customer_id=o.customer_id and o.product_id=p.product_id
group by c.customer_id
;



In [ ]:
Trail 3

select c.customer_id, c.name from Customers c
inner join
(
(
select Distinct(o.customer_id) as custid from Product p, 
Orders o
where p.product_id=o.product_id and LEFT(o.order_date, 7)="2020-06"
group by o.customer_id
having sum(o.quantity*p.price)>=100
)
inner join
(
select Distinct(o.customer_id) as custid from Product p
Orders o
where p.product_id=o.product_id and LEFT(o.order_date, 7)="2020-07"
group by o.customer_id
having sum(o.quantity*p.price)>=100
)
) fog
on fog.custid=c.customer_id
;


In [ ]:
Trial 4

select c.customer_id as customer_id, c.name as name from Customers c
inner join
(
select o.customer_id as custid from Product p
inner join
Orders o
on p.product_id=o.product_id
where o.order_date BETWEEN "2020-06-01" AND "2020-07-31"
group by o.customer_id, o.order_date
having sum(o.quantity*p.price)>=100
) fog

on fog.custid=c.customer_id;

In [ ]:
Wow amazing solution!

select o.customer_id, c.name
from Customers c, Product p, Orders o
where c.customer_id = o.customer_id and p.product_id = o.product_id
group by o.customer_id
having 
(
    sum(case when o.order_date like '2020-06%' then o.quantity*p.price else 0 end) >= 100
    and
    sum(case when o.order_date like '2020-07%' then o.quantity*p.price else 0 end) >= 100
)
